## Apartat 1: detecció de valors absents o no fiables

In [2]:
import pandas as pd
import requests
import numpy as np
from io import StringIO
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

url = "https://raw.githubusercontent.com/tnavarrete-iedib/bigdata-24-25/refs/heads/main/mesuraments-estacio-control-qualitat-aire-illes-balears-estacio-sant-antoni-de-portmany-eivissa.csv"

# Descarregar el contingut
response = requests.get(url)
if response.status_code == 200:
    contingut = StringIO(response.text)
    dades = pd.read_csv(contingut, sep=';')

    # Guardar localment
    dades.to_csv("dades_qualitat_aire.csv", sep=';', index=False)
else:
    print(f"Error al descarregar el fitxer. Codi d'estat: {response.status_code}")

# Apartat 1.1
# Substituim els valors amb flag diferent de 'V' per np.nan
dades.loc[dades['FL_SO2'] != 'V', 'SO2_HI'] = np.nan
dades.loc[dades['FL_NO'] != 'V', 'NO_HI'] = np.nan
dades.loc[dades['FL_NO2'] != 'V', 'NO2_HI'] = np.nan

# Convertim les columnes a tipus numèric
dades['SO2_HI'] = pd.to_numeric(dades['SO2_HI'], errors='coerce')
dades['NO_HI'] = pd.to_numeric(dades['NO_HI'], errors='coerce')
dades['NO2_HI'] = pd.to_numeric(dades['NO2_HI'], errors='coerce')

# Calculam les mitjanes ignorant els valors NaN
mitjana_SO2 = dades['SO2_HI'].mean()
mitjana_NO = dades['NO_HI'].mean()
mitjana_NO2 = dades['NO2_HI'].mean()

print("Apartat 1.1 - Mitjanes després de substituir valors no vàlids per NaN:")
print(f"SO2: {mitjana_SO2:.4f}")
print(f"NO: {mitjana_NO:.4f}")
print(f"NO2: {mitjana_NO2:.4f}")

# Comprovem quants valors vàlids tenim per cada variable
print(f"Nombre de valors vàlids de SO2: {dades['SO2_HI'].notna().sum()} de {len(dades)} ({dades['SO2_HI'].notna().mean()*100:.2f}%)")
print(f"Nombre de valors vàlids de NO: {dades['NO_HI'].notna().sum()} de {len(dades)} ({dades['NO_HI'].notna().mean()*100:.2f}%)")
print(f"Nombre de valors vàlids de NO2: {dades['NO2_HI'].notna().sum()} de {len(dades)} ({dades['NO2_HI'].notna().mean()*100:.2f}%)")

# Apartat 1.2
# Fem una còpia de les dades originals
dades2 = dades.copy()

# Per a SO2, substituim per valor predecessor (ffill)
dades2['SO2_HI'] = dades2['SO2_HI'].fillna(method='ffill')

# Per a NO, substituim per valor successor (bfill)
dades2['NO_HI'] = dades2['NO_HI'].fillna(method='bfill')

# Per a NO2, substituim per la mitjana dels valors no nuls
dades2['NO2_HI'] = dades2['NO2_HI'].fillna(mitjana_NO2)

# Calculam les noves mitjanes
mitjana_SO2_despres = dades2['SO2_HI'].mean()
mitjana_NO_despres = dades2['NO_HI'].mean()
mitjana_NO2_despres = dades2['NO2_HI'].mean()

print("\nApartat 1.2 - Mitjanes després de la substitució:")
print(f"SO2: {mitjana_SO2_despres:.4f}")
print(f"NO: {mitjana_NO_despres:.4f}")
print(f"NO2: {mitjana_NO2_despres:.4f}")

# Comparem amb les mitjanes anteriors
print("\nComparació entre les mitjanes:")
print(f"SO2: {mitjana_SO2:.4f} -> {mitjana_SO2_despres:.4f}, Diferència: {mitjana_SO2_despres - mitjana_SO2:.4f}")
print(f"NO: {mitjana_NO:.4f} -> {mitjana_NO_despres:.4f}, Diferència: {mitjana_NO_despres - mitjana_NO:.4f}")
print(f"NO2: {mitjana_NO2:.4f} -> {mitjana_NO2_despres:.4f}, Diferència: {mitjana_NO2_despres - mitjana_NO2:.4f}")

# Comprovem quants valors tenim després de les substitucions
print(f"\nNombre de valors no NaN de SO2 després de la substitució: {dades2['SO2_HI'].notna().sum()} de {len(dades2)} ({dades2['SO2_HI'].notna().mean()*100:.2f}%)")
print(f"Nombre de valors no NaN de NO després de la substitució: {dades2['NO_HI'].notna().sum()} de {len(dades2)} ({dades2['NO_HI'].notna().mean()*100:.2f}%)")
print(f"Nombre de valors no NaN de NO2 després de la substitució: {dades2['NO2_HI'].notna().sum()} de {len(dades2)} ({dades2['NO2_HI'].notna().mean()*100:.2f}%)")

Apartat 1.1 - Mitjanes després de substituir valors no vàlids per NaN:
SO2: 1.8540
NO: 1.9540
NO2: 4.6365
Nombre de valors vàlids de SO2: 26288 de 117480 (22.38%)
Nombre de valors vàlids de NO: 93153 de 117480 (79.29%)
Nombre de valors vàlids de NO2: 93152 de 117480 (79.29%)

Apartat 1.2 - Mitjanes després de la substitució:
SO2: 2.5128
NO: 2.3081
NO2: 4.6365

Comparació entre les mitjanes:
SO2: 1.8540 -> 2.5128, Diferència: 0.6588
NO: 1.9540 -> 2.3081, Diferència: 0.3541
NO2: 4.6365 -> 4.6365, Diferència: -0.0000

Nombre de valors no NaN de SO2 després de la substitució: 117473 de 117480 (99.99%)
Nombre de valors no NaN de NO després de la substitució: 117480 de 117480 (100.00%)
Nombre de valors no NaN de NO2 després de la substitució: 117480 de 117480 (100.00%)


<ipython-input-2-c9a8c95d49df>:50: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dades2['SO2_HI'] = dades2['SO2_HI'].fillna(method='ffill')
<ipython-input-2-c9a8c95d49df>:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dades2['NO_HI'] = dades2['NO_HI'].fillna(method='bfill')
